# Import Dataset from Roboflow

In [ ]:
#!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="weuUhM15xraEmLMEAiPU")
project = rf.workspace("data-science-3-5kybl").project("helmet-detection-lcyce")
version = project.version(2)
dataset = version.download("yolov7")

# Install Dependencies

In [ ]:
#!pip install pyiqa

In [ ]:
import data
import packages.image_evaluator as image_evaluator
from packages.clahe import create_CLAHE
from tqdm import tqdm
import argparse
import packages.lowlight_test as lowlight_test
import packages.lowlight_train as lowlight_train
import os
import glob
from PIL import Image

dataset = data.dataset('Helmet-Detection-2/train', 'Helmet-Detection-2/test', 'Helmet-Detection-2/valid')
img_eval = image_evaluator.image_evaluator()
all_images = {}
for i, img_path in enumerate(dataset.train_images_path):
    all_images[img_path] = dataset.train_img[i]

for i, img_path in enumerate(dataset.test_images_path):
    all_images[img_path] = dataset.test_img[i]

for i, img_path in enumerate(dataset.valid_images_path):
    all_images[img_path] = dataset.valid_img[i]


In [2]:
save_img_paths = {}
for img_path in all_images.keys():
    save_img_paths[img_path] = {}
    save_img_paths[img_path]['CLAHE'] = img_path.replace('dataset/', 'dataset/CLAHE_processed/')
    save_img_paths[img_path]['ZDCE'] = img_path.replace('dataset/', 'dataset/ZDCE_processed/')

In [3]:
class ImageSaveError(Exception):
   pass

def save_image(image_path, image):
    try:
        image.save(image_path)
    except Exception as e:
        raise ImageSaveError(f"Error saving image: {e}")

# CLAHE Testing

Save CLAHE Images to dataset/CLAHE_processed

In [4]:
for img_path, img in tqdm(all_images.items(), desc=f"Testing Images: ", unit="image"):
    img_zdce = create_CLAHE(img)
    save_image(save_img_paths[img_path]['CLAHE'], img_zdce)

Testing Images: 100%|██████████| 6633/6633 [01:46<00:00, 62.26image/s]


In [5]:
# Test Scores for CLAHE
test_scores = {}
test_scores = {'niqe': [],
                        'psnr': []}
for img in tqdm(dataset.test_img, desc=f"Saving Images for CLAHE: ", unit="image"):
    img_zdce = create_CLAHE(img)
    img_bnw = img_eval.convert_to_grayscale(img)
    niqe_score = img_eval.compute_niqe(img_zdce)
    psnr_score = img_eval.compute_psnr(img_bnw, img_zdce)
    test_scores['niqe'].append(niqe_score)
    test_scores['psnr'].append(psnr_score)


for time in test_scores.keys():
    print(f"NIQE : {sum(test_scores['niqe']) / len(test_scores['niqe'])}")
    print(f"PSNR : {sum(test_scores['psnr']) / len(test_scores['psnr'])}")


Saving Images for CLAHE: 100%|██████████| 279/279 [00:17<00:00, 15.85image/s]

NIQE : 3.0793857445491466
PSNR : tensor([20.0368], device='cuda:0')
NIQE : 3.0793857445491466
PSNR : tensor([20.0368], device='cuda:0')


# Zero-DCE Train and Test (Optional)

In [6]:
# Training of Zero-DCE
args_dict = {
    'lowlight_images_path': "dataset/train/images/",
    'lr': 0.0001,
    'weight_decay': 0.0001,
    'grad_clip_norm': 0.1,
    'num_epochs': 10,
    'train_batch_size': 8,
    'val_batch_size': 4,
    'num_workers': 4,
    'display_iter': 10,
    'snapshot_iter': 10,
    'snapshots_folder': "snapshots/",
    'load_pretrain': False,
    'pretrain_dir': "snapshots/Epoch9.pth"
}

config = argparse.Namespace(**args_dict)

if not os.path.exists(config.snapshots_folder):
    os.mkdir(config.snapshots_folder)

lowlight_train.train(config)

Total training examples: 5805


d:\OneDrive - Mapúa University\Desktop Folder\School Folder\2023-2Term\M2P1\lowlight_train.py:78: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(DCE_net.parameters(),config.grad_clip_norm)


Loss at iteration 10 : 0.5947394371032715
Loss at iteration 20 : 0.570685088634491
Loss at iteration 30 : 0.521859884262085
Loss at iteration 40 : 0.384112149477005
Loss at iteration 50 : 0.6229775547981262
Loss at iteration 60 : 0.3130388855934143
Loss at iteration 70 : 0.4103785455226898
Loss at iteration 80 : 0.258225679397583
Loss at iteration 90 : 0.29635247588157654
Loss at iteration 100 : 0.47787466645240784
Loss at iteration 110 : 0.30879324674606323
Loss at iteration 120 : 0.2851330041885376
Loss at iteration 130 : 0.2806309759616852
Loss at iteration 140 : 0.4583684802055359
Loss at iteration 150 : 0.3029378652572632
Loss at iteration 160 : 0.35152947902679443
Loss at iteration 170 : 0.3611409664154053
Loss at iteration 180 : 0.23215749859809875
Loss at iteration 190 : 0.33556047081947327
Loss at iteration 200 : 0.3035610318183899
Loss at iteration 210 : 0.26950615644454956
Loss at iteration 220 : 0.320589542388916
Loss at iteration 230 : 0.3088638186454773
Loss at iteration 

In [7]:
for img_path, img in tqdm(all_images.items(), desc=f"Saving Images for DCE: ", unit="image"):
    img_zdce = lowlight_test.lowlight(img)
    save_image(save_img_paths[img_path]['ZDCE'], img_zdce)

Saving Images for DCE: 100%|██████████| 6633/6633 [04:16<00:00, 25.89image/s]


In [17]:
# Testing for Zero-DCE
test_results = {}

test_images_path = glob.glob(f"dataset/ZDCE_processed/test/images/*.jpg")
test_result_scores = {}
test_result_scores = {'niqe': [],
                        'psnr': []}

for img_path in tqdm(test_images_path, desc=f"Testing Images: ", unit="image"):
    img = Image.open(img_path)
    img_zdce = lowlight_test.lowlight(img)
    img_bnw = img_eval.convert_to_grayscale(img)
    niqe_score = img_eval.compute_niqe(img_zdce)
    psnr_score = img_eval.compute_psnr(img_bnw, img_zdce)
    test_result_scores['niqe'].append(niqe_score)
    test_result_scores['psnr'].append(psnr_score)

print(f"NIQE : {sum(test_result_scores['niqe']) / len(test_result_scores['niqe'])}")
print(f"PSNR : {sum(test_scores['psnr']) / len(test_scores['psnr'])}")

NIQE : 3.556564442303999
PSNR : tensor([20.0368], device='cuda:0')
